In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.style import use as usestyle
from glob import glob

In [ ]:
usestyle("paperplots.mplstyle")

In [ ]:
from matplotlib import font_manager as fm
fontdir = "/home/ascheb/libfonts/*.ttf"
for fpath in glob(fontdir):
    print(fpath)
    fm.fontManager.addfont(fpath)
from matplotlib import rcParams
rcParams["figure.titlesize"] = 40
rcParams["axes.titlesize"] = 35
rcParams["axes.labelsize"] = 30
rcParams["xtick.labelsize"] = 27
rcParams["ytick.labelsize"] = 27

In [ ]:
afile_control = xr.open_dataset("/moonbow/ascheb/les/2010/hires_control/newprocessdata/mvars-cart-2010-01-02-230000-g1.nc")

In [ ]:
gprops = {"ny": 1098, "nx": 998, "dx": 1000}
xpointsalonglake = np.arange(200, 600, 1)
xalonglake = afile_control["x"].isel(x = xpointsalonglake)
ypointsalonglake = 480+(xpointsalonglake-200)/2.5
yalonglake = ((-np.floor(gprops["ny"]/2)-0.5)+ypointsalonglake)*gprops["dx"]
# sclycoords = np.arange(-(np.ceil(ny/2)-1.5)*dx, (np.floor(ny/2)-0.5)*dx, dx)
yalonglake = np.interp(yalonglake, afile_control["y"], afile_control["y"])
distancealonglake = ((xpointsalonglake-xpointsalonglake[0])**2+(ypointsalonglake-ypointsalonglake[0])**2)**(1/2)
xlakeda = xr.DataArray(coords = {"Distance": distancealonglake, "y": ("Distance", yalonglake), "x": ("Distance", xalonglake.values)}, dims = ["Distance"])

In [ ]:
print(yalonglake[-1]-yalonglake[0])
print(xalonglake[-1]-xalonglake[0])

In [ ]:
alonglakedata = afile_control.interp(x = xlakeda["x"], y = xlakeda["y"], method = "linear")

In [ ]:
print(alonglakedata["LatentHeatFreezingTotal"].longname)

In [ ]:
angle = np.arctan((yalonglake[-1]-yalonglake[0])/(xalonglake[-1]-xalonglake[0]));
alonglakedata["TangentVelocity"] = alonglakedata["u"]*np.cos(angle)+alonglakedata["v"]*np.sin(angle)
alonglakedata["NormalVelocity"] = alonglakedata["v"]*np.cos(angle)-alonglakedata["u"]*np.sin(angle)

In [ ]:
plt.figure(figsize = (15, 3), dpi = 300)
zbnds = slice(0, 60); xbnds = slice(0,400)
plt.axhline(350, color = "black", linestyle = "--")
plt.axhline(500, color = "blacK", linestyle = "--")
plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z=zbnds), np.gradient(alonglakedata["TangentVelocity"].isel(Distance = xbnds, z = zbnds), gprops["dx"], axis = 1), vmin = -5*10**(-3), vmax = 5*10**(-3), cmap = "BrBG")

In [ ]:
plt.figure(figsize = (15, 3), dpi = 300)
zbnds = slice(0, 60); xbnds = slice(0,400)
# plt.axhline(350, color = "black", linestyle = "--")
# plt.axhline(500, color = "blacK", linestyle = "--")
plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z=zbnds), 3*alonglakedata["LatentHeatFreezingTotal"].isel(Distance = xbnds, z = zbnds), vmin = -15, vmax = 15, cmap = "bwr")

In [ ]:
print(alonglakedata["LatentHeatFreezingTotal"].units)

In [ ]:
plt.figure(figsize = (15, 3), dpi = 300)
zbnds = slice(0, 60); xbnds = slice(0,400)
plt.axhline(250, color = "black", linestyle = "--")
plt.axhline(700, color = "blacK", linestyle = "--")
plt.axhline(1000, color = "black", linestyle = "--")
plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z=zbnds), alonglakedata["w"].isel(Distance = xbnds, z = zbnds), vmin = -2, vmax = 2, cmap = "bwr")

In [ ]:
plt.figure(figsize = (15, 3), dpi = 300)
zbnds = slice(0, 60); xbnds = slice(0,400)
plt.axhline(250, color = "black", linestyle = "--")
plt.axhline(700, color = "blacK", linestyle = "--")
plt.axhline(1000, color = "black", linestyle = "--")
plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z=zbnds), alonglakedata["CondMix"].isel(Distance = xbnds, z = zbnds), vmin = 1e-4, vmax = 0.5, cmap = "Blues", norm = "log")

In [ ]:
plt.figure(figsize = (10, 12))
windmp = plt.pcolormesh(afile_control["x"], afile_control["y"], (afile_control["v"].isel(z=slice(5,50)).mean(dim = "z")**2+afile_control["u"].isel(z=slice(5,50)).mean(dim = "z")**2)**(1/2), vmin = 0, vmax = 25, cmap = "jet")
# plt.scatter(xpointsalonglake, ypointsalonglake, color = "green")
# plt.scatter(alonglakedata["x"], alonglakedata["y"], color = "green")
plt.contour(afile_control["x"], afile_control["y"], afile_control["Patch"][1,:,:], levels = [0.999], colors = "black", linewidths = 1)
plt.colorbar(windmp, orientation = "horizontal")

In [ ]:
print(alonglakedata["Distance"])

In [ ]:
plt.figure(figsize = (15, 3), dpi = 300)
zbnds = slice(0, 60); xbnds = slice(0,400)
plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z=zbnds), alonglakedata["TangentVelocity"].isel(Distance = xbnds, z = zbnds)-alonglakedata["TangentVelocity"].isel(Distance = xbnds, z = zbnds).mean(dim = "Distance"), vmin = -5, vmax = 5, cmap = "bwr")

In [ ]:
plt.figure(figsize = (9, 9), dpi = 300)
plt.plot(np.mean(abs(alonglakedata["TangentVelocity"].isel(z = zbnds)-alonglakedata["TangentVelocity"].isel(z=zbnds).rolling({"Distance": 100}).mean()), axis = 1), alonglakedata["z"].isel(z=zbnds))

In [ ]:
plt.figure()
xbnds = slice(175, 225); zbnds = slice(0, 60, 2)
plt.plot(alonglakedata["NormalVelocity"].isel(Distance = xbnds, z = zbnds).mean(dim = "Distance"), alonglakedata["z"].isel(z=zbnds))
print(alonglakedata["NormalVelocity"].isel(Distance = xbnds, z = 16).mean(dim = "Distance"))

In [ ]:
plt.figure(figsize = (15, 9), dpi = 300)
plt.pcolormesh(alonglakedata["Distance"], alonglakedata["z"].isel(z = slice(0, 60)), alonglakedata["w"].isel(z = slice(0, 60)), vmin = -2, vmax = 2, cmap = "bwr")
plt.xticks(np.arange(0, 420, 30))

In [ ]:
plt.figure(figsize = (15, 9))
xbnds = slice(175, 225); zbnds = slice(0, 60, 2)
# plt.pcolormesh(alonglakedata["Distance"], alonglakedata["z"].isel(z = slice(0, 60)), alonglakedata["w"].isel(z = slice(0, 60)), vmin = -2, vmax = 2, cmap = "bwr")
# plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z = zbnds), alonglakedata["VaporMix"].isel(Distance = xbnds, z = zbnds), vmin = 0, vmax = 3, cmap = "BrBG")
normwindmp = plt.pcolormesh(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z=zbnds), alonglakedata["NormalVelocity"].isel(Distance = xbnds, z = zbnds), vmin = -20, vmax = 0, cmap = "jet_r")
plt.quiver(alonglakedata["Distance"].isel(Distance = xbnds), alonglakedata["z"].isel(z = zbnds), alonglakedata["NormalVelocity"].isel(z = zbnds, Distance = xbnds)-alonglakedata["NormalVelocity"].isel(z = zbnds, Distance = xbnds).mean(dim = "Distance"), 3*alonglakedata["w"].isel(z = zbnds, Distance = xbnds), scale = 100, width = 0.002)
plt.colorbar(normwindmp)

In [ ]:
plt.figure(figsize = (10, 12))
windmp = plt.pcolormesh(afile_control["x"], afile_control["y"], (afile_control["v"].isel(z=50)**2+afile_control["u"].isel(z=50)**2)**(1/2), vmin = 0, vmax = 25, cmap = "jet")
# plt.scatter(xpointsalonglake, ypointsalonglake, color = "green")
plt.scatter(alonglakedata["x"], alonglakedata["y"], color = "green")
plt.contour(afile_control["x"], afile_control["y"], afile_control["Patch"][1,:,:], levels = [0.999], colors = "black", linewidths = 1)
plt.colorbar(windmp, orientation = "horizontal")